In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#our feature extractor
model = MobileNetV2(
    weights="imagenet",
    include_top=False, #removes the final classification layer
    pooling="avg"   #outputs a 1280-dimensional vector per image
)

print("Model loaded successfully")


In [ ]:
img_path = r"C:\Users\rawan\Desktop\fruit_veg_classifier\train\fruit\Apple\Image_1.jpg"
img = load_img(img_path, target_size=(224, 224)) # Load image
img_array = img_to_array(img) # Convert to array
img_array = np.expand_dims(img_array, axis=0) # Expand dimensions (model expects batch)
img_array = preprocess_input(img_array) # Preprocess for MobileNetV2
features = model.predict(img_array, verbose=0) # Extract features
print("Feature shape:", features.shape)



In [ ]:
def process_folder(base_path):
    X_features = []  # List to store feature vectors (1280 values per image)
    y_stage1 = [] # List to store stage 1 labels: fruit = 0, vegetable = 1
    y_class = []     # List to store fine-grained class names el homa asamy el folders b asamy el fruit w el vegetable

    for category in ["fruit", "vegetable"]: # Loop over the two main categories akbar folders khales 
        category_path = os.path.join(base_path, category) # Path to either fruit or vegetable folder

        for class_name in os.listdir(category_path):  # Loop over each class inside the category el folders el soghayara gwa el folders el kebera 
            class_path = os.path.join(category_path, class_name) # Full path to the class folder

            if not os.path.isdir(class_path): # Skip if it is not a folder
                continue

            for img_name in os.listdir(class_path): # Loop over all images inside the class folder
                img_path = os.path.join(class_path, img_name) # Full path to the image

                try:
                
                    feature = extract_feature(img_path) # Extract MobileNetV2 feature vector (1280-d)
                    X_features.append(feature) # Store extracted features
                    y_stage1.append(0 if category == "fruit" else 1) # Stage 1 label: 0 for fruit, 1 for vegetable
                    y_class.append(class_name) # Store the class name (Apple, Carrot)

                except Exception as e:
                    print("Error loading:", img_path, e) # Print error if an image fails to load

    return np.array(X_features), np.array(y_stage1), np.array(y_class) # Convert lists to NumPy arrays for ML models

In [ ]:
train_dir = r"C:\Users\rawan\Desktop\fruit_veg_classifier\train"   # Path fel TRAINING FOLDER MISH TESTING LESA

X_train, y_train, y_train_class = process_folder(train_dir) # Extract features and labels from training data

np.save("X_train.npy", X_train) # Save extracted feature vectors
np.save("y_train.npy", y_train) # Save stage 1 labels (fruit vs vegetable)
np.save("y_train_class.npy", y_train_class) # Save class labels (apple, carrot, etc.)

print("Train features saved successfully") # Confirmation message
print("Train X shape:", X_train.shape) # Print shape of feature matrix


In [ ]:
test_dir = r"C:\Users\rawan\Desktop\fruit_veg_classifier\test"  # Path fel TESTING FOLDER

X_test, y_test, y_test_class = process_folder(test_dir) # Extract features and labels from testing data

np.save("X_test.npy", X_test) # Save extracted feature vectors
np.save("y_test.npy", y_test) # Save stage 1 labels (fruit vs vegetable)
np.save("y_test_class.npy", y_test_class) # Save class labels (apple, carrot, etc.)

print("Test features saved successfully") # Confirmation message
print("X_test shape:", X_test.shape) # Print shape of feature matrix


In [ ]:
#STAGE 1: fruit vs vegetable classification
import numpy as np

# Load features
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


In [ ]:
from sklearn.ensemble import RandomForestClassifier # Import Random Forest classifier from scikit-learn -> lIBRARY IMPORTED FO2 KHALES - EL CLASSIFIER EL HANGHAYARHA KAZA MARA 

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Create RF model with 100 trees and fixed random seed
rf_model.fit(X_train, y_train) # Train model on extracted training features and stage 1 labels

print("Stage 1 Random Forest trained successfully")


In [ ]:
#El mehtagenha lel report 3ashan Evalate the performance via computing the accuracy, confusion matrix, …etc.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #metrics for evaluation

y_pred = rf_model.predict(X_test) # Predict stage 1 labels (fruit vs vegetable) on test set

print("Accuracy:", accuracy_score(y_test, y_pred)) # Print overall accuracy
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred)) # Print confusion matrix (true vs predicted)
print("\nClassification Report:\n", classification_report(y_test, y_pred))  # Print precision, recall, f1-score per class



In [ ]:
import joblib

joblib.dump(rf_model, "stage1_rf_model.pkl")
print("Stage 1 model saved successfully")


In [ ]:
#Stage 2 — Classifying fruit types & vegetable types
# Load Stage 1 labels and class names
y_train_class = np.load("y_train_class.npy")
y_test_class = np.load("y_test_class.npy")

# Fruit indices
fruit_idx_train = np.where(y_train==0)[0]
fruit_idx_test = np.where(y_test==0)[0]

# Vegetable indices
veg_idx_train = np.where(y_train==1)[0]
veg_idx_test = np.where(y_test==1)[0]

# Subset features
X_train_fruit = X_train[fruit_idx_train]
y_train_fruit = y_train_class[fruit_idx_train]

X_test_fruit = X_test[fruit_idx_test]
y_test_fruit = y_test_class[fruit_idx_test]

X_train_veg = X_train[veg_idx_train]
y_train_veg = y_train_class[veg_idx_train]

X_test_veg = X_test[veg_idx_test]
y_test_veg = y_test_class[veg_idx_test]

print("Fruit train shape:", X_train_fruit.shape)
print("Vegetable train shape:", X_train_veg.shape)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Train Fruit-Type Classifier
fruit_model = RandomForestClassifier(n_estimators=100, random_state=42)
fruit_model.fit(X_train_fruit, y_train_fruit)

# Evaluate
y_pred_fruit = fruit_model.predict(X_test_fruit)

print("Fruit Accuracy:", accuracy_score(y_test_fruit, y_pred_fruit))
print("\nFruit Confusion Matrix:\n", confusion_matrix(y_test_fruit, y_pred_fruit))
print("\nFruit Classification Report:\n", classification_report(y_test_fruit, y_pred_fruit))

# Save model
joblib.dump(fruit_model, "fruit_type_model.pkl")
print("Fruit-Type model saved ")


In [ ]:
# Train Vegetable-Type Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Train Vegetable-Type Classifier
veg_model = RandomForestClassifier(n_estimators=100, random_state=42)
veg_model.fit(X_train_veg, y_train_veg)

# Evaluate
y_pred_veg = veg_model.predict(X_test_veg)

print("Vegetable Accuracy:", accuracy_score(y_test_veg, y_pred_veg))
print("\nVegetable Confusion Matrix:\n", confusion_matrix(y_test_veg, y_pred_veg))
print("\nVegetable Classification Report:\n", classification_report(y_test_veg, y_pred_veg))

# Save model
joblib.dump(veg_model, "veg_type_model.pkl")
print("Vegetable-Type model saved")


In [ ]:
#TESTING THE FULL PIPELINE ON A SINGLE IMAGE
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
import joblib
import matplotlib.pyplot as plt 

stage1_model = joblib.load("stage1_rf_model.pkl")
fruit_model = joblib.load("fruit_type_model.pkl")
veg_model = joblib.load("veg_type_model.pkl")

feature_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
def extract_feature(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    feature = feature_model.predict(img_array, verbose=0)
    return feature.flatten()

def predict_image(img_path):

    img_display = load_img(img_path)
    plt.imshow(img_display)
    plt.axis('off')
    plt.show()

    features = extract_feature(img_path).reshape(1, -1)
    stage1_pred = stage1_model.predict(features)[0]
    
    if stage1_pred == 0:
        # Fruit
        type_pred = fruit_model.predict(features)[0]
        category = "Fruit"
    else:
        # Vegetable
        type_pred = veg_model.predict(features)[0]
        category = "Vegetable"
    
    print(f"Predicted Category: {category}")
    print(f"Predicted Type: {type_pred}")


img_path = r"C:\Users\rawan\Desktop\fruit_veg_classifier\train\fruit\Apple\Image_1.jpg"
predict_image(img_path)


In [ ]:
#TESTING ON THE WHOLE FOLDER 3ASHAN AKEED HANLA2Y ERRORS - mish kol el swar identified correctly
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
import joblib

#Load models
stage1_model = joblib.load("stage1_rf_model.pkl")
fruit_model = joblib.load("fruit_type_model.pkl")
veg_model = joblib.load("veg_type_model.pkl")

feature_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

#Feature extraction function
def extract_feature(img_path):
    img_array = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    feature = feature_model.predict(img_array, verbose=0)
    return feature.flatten()

#Recursive folder prediction
def predict_all_images(base_folder):
    for root, dirs, files in os.walk(base_folder):
        # Filter image files
        img_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        if len(img_files) == 0:
            continue  # skip empty folders
        
        features_list = []
        for img_file in img_files:
            img_path = os.path.join(root, img_file)
            features_list.append(extract_feature(img_path))
        
        features_batch = np.array(features_list)
        stage1_preds = stage1_model.predict(features_batch)
        
        for i, img_file in enumerate(img_files):
            img_path = os.path.join(root, img_file)
            
            # Display image
            img_display = load_img(img_path)
            plt.imshow(img_display)
            plt.axis('off')
            plt.show()
            
            # Stage 2 prediction
            if stage1_preds[i] == 0:
                type_pred = fruit_model.predict(features_batch[i].reshape(1, -1))[0]
                category = "Fruit"
            else:
                type_pred = veg_model.predict(features_batch[i].reshape(1, -1))[0]
                category = "Vegetable"
            
            print(f"Image: {img_file}")
            print(f"Predicted Category: {category}")
            print(f"Predicted Type: {type_pred}")
            print("-"*40)

#Example usage
predict_all_images(r"C:\Users\rawan\Desktop\fruit_veg_classifier\test")
